In [7]:
import pandas as pd
import plotly.graph_objs as go
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [9]:
r = requests.get(
    "https://www.larimer.org/health/communicable-disease/coronavirus-covid-19/larimer-county-positive-covid-19-numbers"
)
data = []
soup = BeautifulSoup(r.content, 'html.parser')
table = soup.find('table')
rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])  # Get rid of empty values
columns = data[0]
cols = [c.lower().replace(" ", '_') for c in columns]
data = pd.DataFrame(data[1:], columns=cols)
now = datetime.now().isoformat()
data.reported_date = pd.to_datetime(data.reported_date)

In [10]:
data.head()

,reported_date,age_range,gender,city
0,2020-03-09,50s,Female,Johnstown
1,2020-03-15,40s,Male,Fort Collins
2,2020-03-17,50s,Female,Fort Collins
3,2020-03-17,90s,Female,Loveland
4,2020-03-18,40s,Male,Fort Collins


In [11]:
data.describe()

,reported_date,age_range,gender,city
count,42,42,42,42
unique,9,8,2,5
top,2020-03-20 00:00:00,40s,Female,Fort Collins
freq,13,11,23,26
first,2020-03-09 00:00:00,NaN,NaN,NaN
last,2020-03-23 00:00:00,NaN,NaN,NaN


In [12]:
data.city.value_counts()

Fort Collins    26
Loveland        10
Berthoud         3
Timnath          2
Johnstown        1
Name: city, dtype: int64

In [16]:
data[data.city == "Fort Collins"].describe()

,reported_date,age_range,gender,city
count,26,26,26,26
unique,7,6,2,1
top,2020-03-21 00:00:00,40s,Female,Fort Collins
freq,9,7,13,26
first,2020-03-15 00:00:00,NaN,NaN,NaN
last,2020-03-23 00:00:00,NaN,NaN,NaN


In [71]:
# by_day = df.groupby(["city", "reported_date"]).size()
# by_day.index
# by_day.resample('D', level=[0,1]).sum()
x = df.groupby(["city", "reported_date"]).size().reset_index(name='counts')
x.reported_date = pd.to_datetime(x.reported_date)
x.set_index(
    ['reported_date', 'city']
).unstack(
    fill_value=0
).asfreq(
    'D', fill_value=0
).stack().sort_index(level=1).reset_index()

# by_day.index.reported_date.resample("D").sum().fillna(0)
# by_day.reported_date = pd.to_datetime(by_day.reported_date)
# by_day = by_day.sort_index()
# by_day.head()

,reported_date,city,counts
0,2020-03-09,Berthoud,0
1,2020-03-10,Berthoud,0
2,2020-03-11,Berthoud,0
3,2020-03-12,Berthoud,0
4,2020-03-13,Berthoud,0
...,...,...,...
65,2020-03-18,Timnath,0
66,2020-03-19,Timnath,0
67,2020-03-20,Timnath,0
68,2020-03-21,Timnath,0


In [160]:
import plotly.express as px
df = data
layout_overrides = None
groups = ["city", "reported_data"]
x = df.groupby(["city", "reported_date"]).size().reset_index(name='counts')
x.reported_date = pd.to_datetime(x.reported_date)
x.set_index(
    ['reported_date', 'city']
).unstack(
    fill_value=0
).asfreq(
    'D', fill_value=0
).stack().sort_index(level=1).reset_index()

x = x.sort_values(['city', 'reported_date'])
x["cumulative_sum"] = x.groupby('city')['counts'].cumsum()
layout = {
    "title": {"text": f"<b>Total Confirmed COVID-19 Cases by Day</b>"},
    "xaxis": {"automargin": True},
    "yaxis": {
        "automargin": True,
        "title": {"text": "Count"}
    },
}
if layout_overrides:
    layout.update(layout_overrides)
fig = px.scatter(x, x="reported_date", y="cumulative_sum", color="city")
fig.update_layout(layout)
fig.update_traces(mode='lines+markers')
fig.show()

In [153]:
import plotly.express as px
df = data
layout_overrides = None
groups = ["city", "reported_date"]
x = df.groupby(["reported_date"]).size().reset_index(name='counts')
x.reported_date = pd.to_datetime(x.reported_date)
x.head()

x.set_index(
    ['reported_date']
).asfreq(
    'D', fill_value=0
).sort_index(level=1).reset_index()

x["cumulative_sum"] = x['counts'].cumsum()
layout = {
    "title": {"text": f"<b>Total Confirmed COVID-19 Cases by Day</b>"},
    "xaxis": {"automargin": True},
    "yaxis": {
        "automargin": True,
        "title": {"text": "Count"}
    },
}
if layout_overrides:
    layout.update(layout_overrides)
fig = go.Figure(data=go.Scatter(x=x.reported_date, y=x.cumulative_sum, mode='lines+markers'))
fig.show()

In [185]:
import plotly.express as px
df = data
layout_overrides = None
groups = ["city", "reported_data"]
x = df.groupby(["city", "reported_date"]).size().reset_index(name='counts')
x.reported_date = pd.to_datetime(x.reported_date)
x.set_index(
    ['reported_date', 'city']
).unstack(
    fill_value=0
).asfreq(
    'D', fill_value=0
).stack().sort_index(level=1).reset_index()

x = x.sort_values(['city', 'reported_date'])
layout = {
    "title": {"text": f"<b>Total Confirmed COVID-19 Cases by Day</b>"},
    "xaxis": {"automargin": True},
    "yaxis": {
        "automargin": True,
        "title": {"text": "Count"}
    },
}
if layout_overrides:
    layout.update(layout_overrides)
fig = px.scatter(x, x="reported_date", y="counts", color="city")
fig.update_layout(layout)
fig.update_traces(mode='lines+markers')
fig.show()

In [238]:
groups = ["city", "reported_data"]
# x = df.groupby(["city", "age_range"]).size().reset_index(name='counts')

# x.set_index(
#     ['age_range', 'city']
# ).unstack(
#     fill_value=0
# ).stack().sort_index(level=1).reset_index()

x = (df.groupby(["city", "age_range"])['age_range']
.count()
.unstack(fill_value=0)
.stack()
.sort_index(level=0)
.reset_index(name="counts")
)
bars = []
for city in ["Fort Collins", "Loveland"]:
    bar = x[x.city == city]
    print(bar.head())
    bars.append(go.Bar(name=city, x=bar.age_range, y=bar.counts))
fig = go.Figure(data=bars)
fig.update_layout(barmode='group')
fig.update_xaxes(categoryorder="category ascending")
fig.show()

            city age_range  counts
8   Fort Collins       20s       0
9   Fort Collins       30s       7
10  Fort Collins       40s       7
11  Fort Collins       50s       3
12  Fort Collins       60s       3
        city age_range  counts
24  Loveland       20s       1
25  Loveland       30s       0
26  Loveland       40s       1
27  Loveland       50s       1
28  Loveland       60s       1
